In [1]:
from json import dumps,loads
import pandas as pd
from collections import defaultdict

In [2]:
BB_data = pd.read_csv("./data/Hot Stuff.csv")
BB_data

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15
5,http://www.billboard.com/charts/hot-100/2013-0...,1/19/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,15.0,11,16
6,http://www.billboard.com/charts/hot-100/2013-0...,1/26/2013,7,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,7,17
7,http://www.billboard.com/charts/hot-100/2013-0...,2/2/2013,7,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,7.0,7,18
8,http://www.billboard.com/charts/hot-100/2013-0...,2/9/2013,6,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,7.0,6,19
9,http://www.billboard.com/charts/hot-100/2013-0...,2/16/2013,6,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,6.0,6,20


In [3]:
song_art = set()
for i,r in BB_data.iterrows():
    song_art.add((r['Song'],r['Performer']))

In [4]:
from bs4 import BeautifulSoup
import requests
import re

from Levenshtein import ratio

In [5]:
def clean_art_name(art):
    if "Featuring" in art:
        art = art.split("Featuring")[0].strip()
    if "Feat." in art:
        art = art.split("Feat.")[0].strip()
    if "featuring" in art:
        art = art.split("featuring")[0].strip()
    if "&" in art:
        art = art.split("&")[0].strip()
    if "and" in art:
        art = art.split("and")[0].strip()
    if "And" in art:
        art = art.split("And")[0].strip()
    if "With" in art:
        art = art.split("With")[0].strip()
    if "with" in art:
        art = art.split("with")[0].strip()
    if "(" in art:
        art = art.split("(")[0].strip()
    return art
        
def scrape(sa,debug=False):
    s_name,a_name = sa
    if debug:
        print(s_name,a_name)
    url = "https://search.azlyrics.com/search.php"
    headers = {'Cache-Control':'no-cache'}

    # s_name,a_name = "Don't You Worry Child","Swedish House Mafia"
    querystring = {"q":"{} by {}".format(s_name,a_name)}
    resp = requests.request("GET",url,headers=headers,params=querystring)


    list_of_results_reg = r'''<div class="container main-page">\r\n    <div class="row">\r\n        <div class="col-xs-12 col-sm-10 col-sm-offset-1 col-md-8 col-md-offset-2 text-center" style="padding-top: 20px;">\r\n            \r\n            <div class="panel">\r\n                <div class="panel-heading"><b>Song results:</b><br><small>\[.*\]</small></div>\r\n                <table class="table table-condensed">\r\n                    \r\n                    \r\n            ([\s\S]*)\r\n            \r\n                    \r\n                </table>\r\n            </div>'''
    m = re.search(list_of_results_reg, resp.text)
    if not m:
        querystring = {"q":"{} by {}".format(s_name,clean_art_name(a_name))}
        resp = requests.request("GET",url,headers=headers,params=querystring)
        m = re.search(list_of_results_reg, resp.text)
    if m:
        rows = m.group(1)
        search_data_reg = r'''<tr><td class="text-left visitedlyr">\s*\d{1,2}\. <a href="(https://www\.azlyrics\.com/lyrics/[\w-]+/[\w-]+\.html)" target="_blank"><b>([\s\S]*?)</b></a>  by <b>([\s\S]*?)</b><br/>\s*<small>[\s\S]*?</small>\s*</td></tr>'''
        matches = re.search(search_data_reg,rows)

        soup = BeautifulSoup(rows, 'html.parser')
        result_list = []
        for r in soup.find_all('tr'):
            if debug:
                print(str(r))
            match = re.search(search_data_reg,str(r))
            res = "{} by {}".format(match.group(2),match.group(3))
            orig = querystring['q']
            if debug:
                print(orig,"=?=",res)
            result_list.append((ratio(res.lower(),orig.lower()),res,orig,match.group(1),s_name, a_name))
        return sorted(result_list,reverse=True)
    else:
        return []

In [14]:
scraped_data = {}
cnt = 0
song_art_list = list(song_art)
for i in range(len(song_art_list)):
    s,a = song_art_list[i]
    try:
        scraped_data[(s,a)] = scrape((s,a))
    except Exception as e:
        print("exception at",i,s,a)
        raise e
    cnt+=1
    if cnt%100 == 0:
        print("{:4.3f}% completed".format(cnt/len(song_art)*100))

0.358


KeyboardInterrupt: 

In [8]:
scrape(("Don't You Worry Child","Swedish House Mafia"))

[(1.0,
  "Don't You Worry Child by Swedish House Mafia",
  "Don't You Worry Child by Swedish House Mafia",
  'https://www.azlyrics.com/lyrics/swedishhousemafia/dontyouworrychild.html',
  "Don't You Worry Child",
  'Swedish House Mafia'),
 (0.7674418604651163,
  "Don't You Worry Child by Within Temptation",
  "Don't You Worry Child by Swedish House Mafia",
  'https://www.azlyrics.com/lyrics/withintemptation/dontyouworrychild.html',
  "Don't You Worry Child",
  'Swedish House Mafia'),
 (0.7532467532467533,
  "Don't You Worry Child by Sam Tsui",
  "Don't You Worry Child by Swedish House Mafia",
  'https://www.azlyrics.com/lyrics/samtsui/dontyouworrychild.html',
  "Don't You Worry Child",
  'Swedish House Mafia'),
 (0.7228915662650602,
  "Don't You Worry Child by Madilyn Bailey",
  "Don't You Worry Child by Swedish House Mafia",
  'https://www.azlyrics.com/lyrics/madilynbailey/dontyouworrychild.html',
  "Don't You Worry Child",
  'Swedish House Mafia'),
 (0.7160493827160493,
  "Don't You W

In [15]:
print(len(scraped_data))

109


In [13]:
scrape(("Lil' Red Riding Hood","Sam The Sham and the Pharaohs"),debug=True)

Lil' Red Riding Hood Sam The Sham and the Pharaohs


[]

In [ ]:
import multiprocessing

mp_scraped_data = {}
cnt = 0
song_art_list = list(song_art)[:20]

with multiprocessing.Pool(processes=multiprocessing.cpu_count()*4) as pool:
    for res_ls in pool.imap_unordered(scrape,song_art_list):
        mp_scraped_data[(res_ls[6],res_ls[7])] = res_ls
        cnt+=1
        if cnt%100 == 0:
            print("{:4.3f}".format(cnt/len(song_art)*100))

In [4]:
len(song_art)

27949

In [10]:
#map(lambda d: ,BB_data['WeekID'])
def date(s):
    m = re.search(r'^(\d{1,2})\/(\d{1,2})\/(\d{4})$',s)
    return int(m.group(3)),int(m.group(1)),int(m.group(2))
date('12/15/2012')
print(min(map(date,BB_data['WeekID'])))
print(max(map(date,BB_data['WeekID'])))

(1958, 8, 2)
(2018, 12, 29)
